In [ ]:
import pandas as pd




"""
    Для определения наличия сделки нужно добавить в выгрузку поле 'статус закрытия заявки' и выбрать только значения 'yes' 
    Еще нужно проверить даты окончания последних и незакрытых сессий!!!!





    Описание:
        Из общего датафрейма со всеми клиентами создается датафрейм со всеми сеансами по следующему принципу: 

            - По каждому клиенту (поле 'parent_indx') формируется список всех его заявок. Дата его первого сеанса определяется от 
              даты создания самой ранней заявки до прошествия 180 дней (или до даты сделки, если она состоялась менее, чем за 180 дней). 
              Если после окончания первого сеанса у клиента есть еще заявки, то они группируются аналогичным образом в следующий сеанс и так далее.

            - Все собранные сеансы заносятся в соответствующий датафрейм 
"""



df_input = pd.read_csv('datasets/tymen.csv')
df = df_input.copy()


# датафрейм с сессиями
df_sessions = pd.DataFrame(
    {
        'parent_indx': [],      # id клиента
        'session_num': [],       # id сессии
        'session_start': [],    # дата начала сессии (дата создания первой заявки в сессии)
        'session_end': [],      # дата окончания сессии (дата создания первой заявки в сессии + 180 дней (или дата сделки, если есть))
        'tickets': [],          # заявки из сессии
        'deal': [],             # есть сделка?
    }
)

unique_clients = list(df['parent_indx'].unique())

# пример для parent_indx = 11
for client in unique_clients: 

    # все заявки по клиенту
    df_client_all_tickets = df[df['parent_indx'] == client]

    # все сессии клиента
    df_client_all_sessions = pd.DataFrame()

    # номер сессии клиента
    session_num = 1

    # цикл группировки заявок клиента по сессиям
    while len(df_client_all_tickets) > 0:

        session_start = df_client_all_tickets['дата создания заявки на покупку/продажу'].min()
        session_end = str(pd.to_datetime(df_client_all_tickets['дата создания заявки на покупку/продажу'].min()) + pd.DateOffset(days=180))
        deal = False


        df_client_session = df_client_all_tickets[
            (df_client_all_tickets['дата создания заявки на покупку/продажу'] >= session_start) &
            (df_client_all_tickets['дата создания заявки на покупку/продажу'] <= session_end)
        ]

        # проверка на наличие сделок в сессии
        deals_in_session = list(df_client_session[~df_client_session['дата закрытия заявки на покупку/продажу'].isna()]['дата закрытия заявки на покупку/продажу'])
        if deals_in_session:
            deal = True
            session_end = min(deals_in_session)
            df_client_session = df_client_session[
                df_client_session['дата создания заявки на покупку/продажу'] < session_end
            ]

        # заявки в сессии
        tickets = list(df_client_session['Номер заявки на покупку/продажу'])

        df_session = pd.DataFrame(
            {
                'parent_indx': client,  
                'session_num': session_num,   
                'session_start': session_start,
                'session_end': session_end,  
                'tickets': [tickets],      
                'deal': deal,         
            }
        )
        df_client_all_sessions = pd.concat([df_client_all_sessions, df_session])


        df_client_all_tickets = df_client_all_tickets[~df_client_all_tickets['Номер заявки на покупку/продажу'].isin(tickets)]
        session_num += 1

    df_sessions = pd.concat([df_sessions, df_client_all_sessions])
 
    
df_sessions




In [ ]:
df[df['parent_indx'] == 11]